In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf gcolab.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown ### Download Model
#@markdown Select a Model from the dropdown and run this cell to download the files.

Model = "AbyssOrangeMix3" #@param ["Custom", "AbyssOrangeMix3", "OrientalMix", "Counterfeit2.5 2.5D Fix"]

#@markdown If you select Custom,
ModelLink = "" #@param {type:"string"}
VaeLink = "" #@param {type:"string"}
Safe_Tensors = True #@param {type:"boolean"}
Temp_Storage = True #@param {type:"boolean"}

if Model == "Custom" and ModelLink == "":
  print("Please select an Model from the dropdown.")
if Model != "Custom" and ModelLink != "" and VaeLink != "":
  print ("You cannot do this.")
else:
  # create folders if they don't exist
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE
  
  # define the download links
  file_extension = '.safetensors' if Safe_Tensors else '.ckpt'
  file_type = "SafeTensor" if Safe_Tensors else "PickleTensor"
  StorageType = '/content/' if Temp_Storage else '/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/'
  %cd $StorageType
  clear_output()
  downloads = {
      'Custom': [
        (ModelLink, StorageType + "model" + file_extension),
        (VaeLink if VaeLink else "https://huggingface.co/Yukihime256/840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE/model.vae.pt")
      ],
      'AbyssOrangeMix3': [
          ("https://civitai.com/api/download/models/11811?type=Model&format=" +file_type, StorageType + "model.safetensors"),
          ("https://civitai.com/api/download/models/11811?type=VAE&format=Other", "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE/model.vae.pt")
      ],
      'OrientalMix': [
          ("https://civitai.com/api/download/models/19348?type=Model&format=" +file_type, StorageType + "model.safetensors"),
          ("https://huggingface.co/Yukihime256/840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE/model.vae.pt")
      ],
      'Counterfeit2.5 2.5D Fix': [
          ("https://civitai.com/api/download/models/12408?type=Model&format=" +file_type, StorageType + "model.ckpt"),
          ("https://huggingface.co/Yukihime256/840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/VAE/model.vae.pt")
      ]
  }
  # download the files for the selected Model
  if Model in downloads:
    print('[1;32mDownloading Model and VAE...')
    for url, filepath in downloads[Model]:
      complete = url, filepath
      output_str = f'"{complete[1]}" "{complete[0]}"'
      !gdown --fuzzy -O {output_str}
  else:
    print("Invalid selection.")

model = StorageType

clear_output()
inf('\u2714 Done','success', '50px')


In [ ]:
#@markdown # Other stuff
Extras = True #@param {type:"boolean"}


if Extras:
    # Create Lora Folder
    get_ipython().system('mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora')
    clear_output()

    # EasyNegative Embedding
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/embeddings/EasyNegative.safetensors https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors
    clear_output()
    
    clear_output()
    # Lora dump:
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/JpDollLikeness.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Japanese-doll-likeness.safetensors
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/KrDollLikeness.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Korean-doll-likeness.safetensors
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/TwDollLikeness.safetensors https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Japanese-doll-likeness.safetensors
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/BiCBetterBodies.safetensors https://civitai.com/api/download/models/13300
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/ShirtLift.safetensors https://civitai.com/api/download/models/7870
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/BiCV.safetensors https://civitai.com/api/download/models/12873
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/StLouisAzureLane.safetensors https://civitai.com/api/download/models/7840?type=Model&format=SafeTensor
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/BonGlass.safetensors https://civitai.com/api/download/models/8614
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/HiPoly3DLora.safetensors https://civitai.com/api/download/models/10301
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/MiddleFinger.safetensors https://civitai.com/api/download/models/8244
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/FG-Small.safetensors https://civitai.com/api/download/models/8217
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/InvertedNips.safetensors https://civitai.com/api/download/models/9618
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/SlingshotBikini.safetensors https://civitai.com/api/download/models/7985
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/NipPiercingChains.safetensors https://civitai.com/api/download/models/11896
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/CSRB-Style.safetensors https://civitai.com/api/download/models/13223
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/MoXin.safetensors https://civitai.com/api/download/models/14856
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/AkihikoYoshida.safetensors https://civitai.com/api/download/models/6469
    clear_output()
    
    # Character Loras
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/MashiroShiina.safetensors https://civitai.com/api/download/models/13359
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/Makima.safetensors https://civitai.com/api/download/models/6244
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/Narmaya.safetensors https://civitai.com/api/download/models/13838
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/Nilou.safetensors https://civitai.com/api/download/models/6238
    clear_output()
    
    !wget -N --checksum -nc -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Lora/ElegantHanfu.safetensors https://civitai.com/api/download/models/9470
    
    clear_output()
    inf('\u2714 Done','success', '50px')
else:
    clear_output()
    inf('\u2714 Done','success', '50px')


In [ ]:
#@markdown # ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse

Model = "None" #@param [ "None", "All", "Canny", "Depth", "HED", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "T2iadapter_Models"]

#@markdown - Download/update ControlNet extension and its models.

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
Depth='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
HED='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
MLSD='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
Normal='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
OpenPose='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
Scribble='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
Seg='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'


with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git pull
    %cd /content

!cp /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/*.yaml /content/gdrive/MyDrive/sd/stable-diffusion-webui/models
mdldir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models"

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt
with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()
!rm CN_models.txt

if Model == "All": 
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

elif Model == "None":
  clear_output()
  inf('\u2714 Skipped','info', '50px')
  
elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

else:
  download(globals()[Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Start Stable-Diffusion
from IPython.utils import capture
import time
import sys
import fileinput

Use_localtunnel = False #@param {type:"boolean"}

User = "" #@param {type:"string"}
Password= "" #@param {type:"string"}
#@markdown - Add credentials to your Gradio interface (optional).

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.9/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.9/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars

###### **Notebook From https://github.com/TheLastBen/fast-stable-diffusion, if you encounter any issues, feel free to discuss them.** [Support](https://ko-fi.com/thelastben)